## Prediction
### Deployment of kserve will take few minutes to comes to ready state, so wait for few minutes before actually executing the below cell.

In [ ]:
import requests
EZAF_ENV = "hpe-qa1-ezaf" 
token_url = f"https://keycloak.{EZAF_ENV}.com/realms/UA/protocol/openid-connect/token"

data = {
    "username" : input("Enter username."),
    "password" : input("Enter password."),
    "grant_type" : "password",
    "client_id" : "ua-grant",
}

token_responce = requests.post(token_url, data=data, allow_redirects=True, verify=False)

token = token_responce.json()["access_token"]

In [ ]:
NAMESPACE = "hpedemo-user01"
DEPLOYMENT_NAME = "bike-sharing-1"
MODEL_NAME = DEPLOYMENT_NAME
SVC = f'{DEPLOYMENT_NAME}-predictor-default.{NAMESPACE}.{EZAF_ENV}.com'
URL = f"https://{SVC}/v2/models/{MODEL_NAME}/infer"

print(URL)

names = ['season', 'year', 'month', 'hour_of_day', 'is_holiday', 'weekday', 'is_workingday', 
         'weather_situation', 'temperature', 'feels_like_temperature', 'humidity', 'windspeed']

input_data = [
    [1, 2, 1, 0, 0, 6, 0, 1, 0.24, 0.2879, 0.81, 0.0000],
    [1, 5, 1, 0, 0, 6, 1, 1, 0.24, 0.2879, 0.81, 0.0000]
]

inputs = {
  "inputs": [
    {
      "name": "ndarray",
      "shape": [2, 12],
      "datatype": "FP32",
      "data": input_data
    }
  ]
}

headers = {"Authorization": f"Bearer {token}"}

response = requests.post(URL, json=inputs, headers=headers, verify=False)

print(response.reason)

output = response.json()['outputs'][0]['data']

print("Rendted Bikes Per Hours:\n")
for item, out in zip(input_data, output):
    input_dict = dict(zip(names,item))
    print(f"Input Data: {input_dict} \n\nBike Per Hour: {out}\n")